In [1]:
from chessbot import choose_move
from massimobot import sexy_play_method
import chess
import numpy as np
from random import randrange

In [2]:
def generateStartingPositions(n):
    boards=np.zeros(n, dtype=chess.Board)
    moves=np.zeros(n*2, dtype=np.ndarray)
    for i in range(n):
        board=chess.Board()
        for j in range(4):
            legalmoves=list(board.legal_moves)
            move=board.san(legalmoves[randrange(len(legalmoves))])
            if j==0:
                moves[2*i]=move
                moves[2*i+1]=move
            else:
                moves[2*i]=np.append(moves[2*i],move)
                moves[2*i+1]=np.append(moves[2*i+1],move)
            board.push_san(move)
        boards[i]=board
    return [boards,moves]

In [49]:
import chess.engine
def addEval(engine, board, evaluation, i):
    #Engine from white POV
    tmpeval=engine.analyse(board, chess.engine.Limit(time=0.1))['score'].pov(1)
    if tmpeval.is_mate():
        if tmpeval.mate()<0:
            evaluation[i].append(-2000)
        else:
            evaluation[i].append(2000)
    else:
        evaluation[i].append(tmpeval.score())
    return evaluation

In [55]:
def pit(n):
    """
    Input: 
        n: half of games to be played
    
    Output:
        record: record of games, see comment below
        moves: Array of moves played
        evaluation: evaluation of position from white perspective in CP (mate = 2000)
        colorWin: Array of length n*2 describing outcome of each game
        0: draw, 1: white win, -1: black win
    """
    engine=chess.engine.SimpleEngine.popen_uci("stockfish/14/bin/stockfish")
    [boards, moves]=generateStartingPositions(n)
    colorWin=np.zeros(n*2)
    evaluation={}
    for i in range(n*2):
        evaluation[i]=[29]
    record=np.zeros(3) #[Grant wins, Massimo wins, draws]
    for i in range(n):
        board=boards[i].copy() #Grant is white
        while board.outcome() is None:
            grantmove=board.san(choose_move(board,1))
            board.push_san(grantmove)
            evaluation=addEval(engine, board, evaluation, i*2)
            moves[i*2]=np.append(moves[i*2],grantmove)
            if board.outcome() is not None:
                break
            massimomove=board.san(sexy_play_method(board,board.turn))
            board.push_san(massimomove)
            evaluation=addEval(engine, board, evaluation, i*2)
            moves[i*2]=np.append(moves[i*2],massimomove)
        if board.outcome().winner is None:
            record[2]+=1
        elif board.outcome().winner==True:
            colorWin[i*2]=1
            record[0]+=1
        else:
            colorWin[i*2]=-1
            record[1]+=1
        board=boards[i] #Massimo is white
        while board.outcome() is None:
            massimomove=board.san(sexy_play_method(board,board.turn))
            board.push_san(massimomove)
            evaluation=addEval(engine, board, evaluation, i*2+1)
            moves[i*2+1]=np.append(moves[i*2+1],massimomove)
            if board.outcome() is not None:
                break
            grantmove=board.san(choose_move(board,1))
            board.push_san(grantmove)
            evaluation=addEval(engine, board, evaluation, i*2+1)
            moves[i*2+1]=np.append(moves[i*2+1],grantmove)
        if board.outcome().winner is None:
            record[2]+=1
        elif board.outcome().winner==True:
            colorWin[i*2+1]=1
            record[1]+=1
        else:
            colorWin[i*2+1]=1
            record[0]+=1
        print(i)
    return [record,moves, evaluation, colorWin]

In [51]:
[record, moves, evaluation, colorWin]=pit(1)

0


In [85]:
def comeback(evaluation, colorWin, n):
    if n >= len(colorWin):
        print("n must be smaller than number of games played")
        return
    # n is number of games to find comebacks in
    winnerEval=[]
    for i in range(len(colorWin)):
        winnerEval.append(sum(evaluation[i])*colorWin[i])
    return np.argpartition(winnerEval, n)[:n]

In [87]:
comeback(evaluation, colorWin, 1)

array([1])